In [1]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('./dataset/resumo_anual_2019.csv', delimiter=";", encoding='latin-1')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37968 entries, 0 to 37967
Data columns (total 38 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   EMPRESA (SIGLA)                    37968 non-null  object 
 1   EMPRESA (NOME)                     37968 non-null  object 
 2   EMPRESA (NACIONALIDADE)            37968 non-null  object 
 3   ANO                                37968 non-null  int64  
 4   MÊS                                37968 non-null  int64  
 5   AEROPORTO DE ORIGEM (SIGLA)        37959 non-null  object 
 6   AEROPORTO DE ORIGEM (NOME)         37959 non-null  object 
 7   AEROPORTO DE ORIGEM (UF)           31061 non-null  object 
 8   AEROPORTO DE ORIGEM (REGIÃO)       31064 non-null  object 
 9   AEROPORTO DE ORIGEM (PAÍS)         37959 non-null  object 
 10  AEROPORTO DE ORIGEM (CONTINENTE)   37959 non-null  object 
 11  AEROPORTO DE DESTINO (SIGLA)       37968 non-null  obj

In [4]:
df.head()

,EMPRESA (SIGLA),EMPRESA (NOME),EMPRESA (NACIONALIDADE),ANO,MÊS,AEROPORTO DE ORIGEM (SIGLA),AEROPORTO DE ORIGEM (NOME),AEROPORTO DE ORIGEM (UF),AEROPORTO DE ORIGEM (REGIÃO),AEROPORTO DE ORIGEM (PAÍS),...,COMBUSTÍVEL (LITROS),DISTÂNCIA VOADA (KM),DECOLAGENS,CARGA PAGA KM,CARGA GRATIS KM,CORREIO KM,ASSENTOS,PAYLOAD,HORAS VOADAS,BAGAGEM (KG)
0,AAF,AIGLE AZUR,ESTRANGEIRA,2019,1,LFPO,ORLY (NEAR PARIS),NaN,NaN,FRANÇA,...,NaN,149856.0,16.0,920725000.0,0.0,0.0,4592.0,770089.0,"144,86",NaN
1,AAF,AIGLE AZUR,ESTRANGEIRA,2019,1,SBKP,CAMPINAS,SP,SUDESTE,BRASIL,...,NaN,149856.0,16.0,263700000.0,25232000.0,0.0,4592.0,770089.0,"227,34",NaN
2,AAF,AIGLE AZUR,ESTRANGEIRA,2019,2,LFPO,ORLY (NEAR PARIS),NaN,NaN,FRANÇA,...,NaN,149856.0,16.0,617173000.0,0.0,0.0,4592.0,770089.0,"107,35",NaN
3,AAF,AIGLE AZUR,ESTRANGEIRA,2019,2,SBKP,CAMPINAS,SP,SUDESTE,BRASIL,...,NaN,149856.0,16.0,0.0,0.0,0.0,4592.0,770089.0,"267,29",NaN
4,AAF,AIGLE AZUR,ESTRANGEIRA,2019,3,LFPO,ORLY (NEAR PARIS),NaN,NaN,FRANÇA,...,NaN,159222.0,17.0,933032000.0,0.0,0.0,4879.0,1252270.0,"134,73",NaN


In [5]:
df.columns = ['EMPRESA_SIGLA','EMPRESA_NOME','EMPRESA_NACIONALIDADE','ANO','MES','AEROPORTO_DE_ORIGEM_SIGLA','AEROPORTO_DE_ORIGEM_NOME','AEROPORTO_DE_ORIGEM_UF','AEROPORTO_DE_ORIGEM_REGIAO','AEROPORTO_DE_ORIGEM_PAIS','AEROPORTO_DE_ORIGEM_CONTINENTE','AEROPORTO_DE_DESTINO_SIGLA','AEROPORTO_DE_DESTINO_NOME','AEROPORTO_DE_DESTINO_UF','AEROPORTO_DE_DESTINO_REGIAO','AEROPORTO_DE_DESTINO_PAIS','AEROPORTO_DE_DESTINO_CONTINENTE','NATUREZA','GRUPO_DE_VOO','PASSAGEIROS_PAGOS','PASSAGEIROS_GRATIS','CARGA_PAGA_KG','CARGA_GRATIS_KG','CORREIO_KG','ASK','RPK','ATK','RTK','COMBUSTIVEL_LITROS','DISTANCIA_VOADA_KM','DECOLAGENS','CARGA_PAGA_KM','CARGA_GRATIS_KM','CORREIO_KM','ASSENTOS','PAYLOAD','HORAS_VOADAS','BAGAGEM_KG']
df['HORAS_VOADAS'] = df['HORAS_VOADAS'].str.replace(',', '.').astype(float)

In [6]:
df = df[~(df[['EMPRESA_SIGLA','AEROPORTO_DE_ORIGEM_SIGLA','AEROPORTO_DE_DESTINO_SIGLA']].duplicated(keep=False)) & (df['AEROPORTO_DE_ORIGEM_SIGLA'] != df['AEROPORTO_DE_DESTINO_SIGLA'])]

In [7]:
columns = ['EMPRESA_SIGLA','EMPRESA_NOME','AEROPORTO_DE_ORIGEM_SIGLA','AEROPORTO_DE_ORIGEM_NOME','AEROPORTO_DE_ORIGEM_UF','AEROPORTO_DE_DESTINO_SIGLA','AEROPORTO_DE_DESTINO_NOME','AEROPORTO_DE_DESTINO_UF','DECOLAGENS','HORAS_VOADAS']
df = df[columns].dropna()
df

,EMPRESA_SIGLA,EMPRESA_NOME,AEROPORTO_DE_ORIGEM_SIGLA,AEROPORTO_DE_ORIGEM_NOME,AEROPORTO_DE_ORIGEM_UF,AEROPORTO_DE_DESTINO_SIGLA,AEROPORTO_DE_DESTINO_NOME,AEROPORTO_DE_DESTINO_UF,DECOLAGENS,HORAS_VOADAS
105,AAL,"AMERICAN AIRLINES, INC.",SBSN,SANTARÉM,PA,SBEG,MANAUS,AM,1.0,0.32
276,ACL,AER CARAIBE S.A.,SBEG,MANAUS,AM,SBKP,CAMPINAS,SP,1.0,4.42
278,ACL,AER CARAIBE S.A.,SBKP,CAMPINAS,SP,SBEG,MANAUS,AM,2.0,7.00
292,ADB,ANTONOV AIRLINES,SBSG,SÃO GONÇALO DO AMARANTE,RN,SBGL,RIO DE JANEIRO,RJ,1.0,4.00
419,AFR,SOCIÉTÉ AIR FRANCE,SBRF,RECIFE,PE,SBGR,GUARULHOS,SP,1.0,3.60
...,...,...,...,...,...,...,...,...,...,...
37330,TTL,TOTAL LINHAS AÉREAS S.A.,SBSL,SÃO LUÍS,MA,SBCF,CONFINS,MG,1.0,2.82
37337,TTL,TOTAL LINHAS AÉREAS S.A.,SBVT,VITÓRIA,ES,SBSL,SÃO LUÍS,MA,1.0,2.57
37776,UAL,"UNITED AIRLINES, INC",SBGL,RIO DE JANEIRO,RJ,SBEG,MANAUS,AM,1.0,4.10
37782,UAL,"UNITED AIRLINES, INC",SBPJ,PALMAS,TO,SBGR,GUARULHOS,SP,1.0,2.23
